In [1]:
%%capture
!pip install datasets

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [3]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Crear Dataset de wordlist
#"""
import pandas as pd
import gzip
import random

# Ruta base
base_path = '/content/drive/MyDrive/Train Family/'

# Familias basadas en listas de palabras
wordlist_families = ['suppobox', 'charbot', 'gozi', 'matsnu', 'manuelita', 'rovnix', 'deception', 'nymaim']

# Inicializamos una lista para guardar los DataFrames
dfs = []

# Para cada familia
for family in wordlist_families:
    file_path = base_path + family + '.gz'
    with gzip.open(file_path, 'rt') as f:
        df = pd.read_csv(f)

    # Nos quedamos solo con las columnas deseadas
    df = df[['domain', 'family', 'label']]

    # Muestreamos 18000 dominios aleatoriamente
    df_sampled = df.sample(n=10000, random_state=42)

    dfs.append(df_sampled)

# Ahora cargamos los dominios normales (legit.gz)
file_path_legit = base_path + 'legit.gz'
with gzip.open(file_path_legit, 'rt') as f:
    df_legit = pd.read_csv(f)

df_legit = df_legit[['domain', 'family', 'label']]

# Muestreamos 18000 normales
df_legit_sampled = df_legit.sample(n=80000, random_state=42)

dfs.append(df_legit_sampled)

# Unimos todo
train_wl = pd.concat(dfs).reset_index(drop=True)

# Verificamos
print(train_wl['family'].value_counts())
print(train_wl.head())

# Si querés guardar el dataset resultante:
train_wl.to_csv('/content/drive/MyDrive/MOE_DGA/train_wl.csv', index=False)
#"""

family
legit        80000
suppobox     10000
charbot      10000
matsnu       10000
gozi         10000
manuelita    10000
rovnix       10000
deception    10000
nymaim       10000
Name: count, dtype: int64
                  domain    family label
0    nailconsiderable.ru  suppobox   dga
1       stilldelight.net  suppobox   dga
2  kimberleekatheryn.net  suppobox   dga
3           soilbeen.net  suppobox   dga
4          visitform.net  suppobox   dga


In [5]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-30 15:47:22 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 04-30 15:47:22 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.3: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.5.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [6]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [7]:
import pandas as pd
from datasets import Dataset
import json

# Carga tu dataset desde CSV
file_path = '/content/drive/My Drive/MOE_DGA/train_wl.csv'
df = pd.read_csv(file_path)

# Crear conversaciones a partir de input_text y Label
conversations = []
for _, row in df.iterrows():
    domain = row['domain']
    label = row['label']

    input_text = f"Classify the following domain name as dga or normal: {domain}"
    # Crear respuesta ficticia basada en Label
    if label == "dga":
        response = "The domain name is classified as dga."
    else:
        response = "The domain name is classified as normal."

    # Crear estructura de conversación
    conversation = {
        "conversations": [
            {"from": "human", "value": input_text},
            {"from": "gpt", "value": response}
        ]
    }
    conversations.append(conversation)

# Convertir a Hugging Face Dataset
dataset = Dataset.from_list(conversations)

In [8]:
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
from datasets import Dataset

from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

# Función para formatear prompts
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Estandarizar el dataset (ShareGPT)
dataset = standardize_sharegpt(dataset)

# Mapear el formato para aplicar la plantilla del tokenizador
dataset = dataset.map(formatting_prompts_func, batched=True)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/160000 [00:00<?, ? examples/s]

Map:   0%|          | 0/160000 [00:00<?, ? examples/s]

In [9]:
dataset[0]["text"]

'<bos><start_of_turn>user\nClassify the following domain name as dga or normal: nailconsiderable.ru<end_of_turn>\n<start_of_turn>model\nThe domain name is classified as dga.<end_of_turn>\n'

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 128,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        dataset_num_proc=2,
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/160000 [00:00<?, ? examples/s]

In [11]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/160000 [00:00<?, ? examples/s]

In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 160,000 | Num Epochs = 1 | Total steps = 312
O^O/ \_/ \    Batch size per device = 128 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (128 x 4 x 1) = 512
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,8.738100
2,8.723900
3,7.970300
4,5.556800
5,4.114700
6,3.003800
7,1.469800
8,0.492300
9,0.120900
10,0.107500


Step,Training Loss
1,8.738100
2,8.723900
3,7.970300
4,5.556800
5,4.114700
6,3.003800
7,1.469800
8,0.492300
9,0.120900
10,0.107500


In [ ]:
# Guardar el modelo y el tokenizador
trainer.model.save_pretrained("outputs/model")
tokenizer.save_pretrained("outputs/model")


('outputs/model/tokenizer_config.json',
 'outputs/model/special_tokens_map.json',
 'outputs/model/tokenizer.json')

In [13]:
# Definimos la ruta donde queremos guardar
save_directory = "/content/drive/MyDrive/MOE_DGA/outputs_gemma/model"

# Guardamos el modelo y el tokenizador
trainer.model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


['/content/drive/MyDrive/MOE_DGA/outputs_gemma/model/processor_config.json']

In [14]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\nThe sequence is classified as the Fibonacci sequence.<end_of_turn>']

In [15]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Classify the following domain name as dga or normal: visitform.net",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 10, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The domain name is classified as dga.<end_of_turn>


In [ ]:
import requests
import pandas as pd
import numpy as np
import time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import sys
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import re

families = [
    #'matsnu.gz'#,
    #'suppobox.gz',
    #'charbot.gz',
    #'gozi.gz',
    'manuelita.gz'#,
    #'rovnix.gz',
    #'deception.gz',
    #'nymaim.gz'
]

runs = 30

for family in families:
    print(family)
    dga = pd.read_csv(f'/content/drive/My Drive/Familias_Test/{family}', chunksize=50)
    legit = pd.read_csv('/content/drive/My Drive/Familias_Test/legit.gz', chunksize=50)
    dfs = []
    for run in range(runs):
        print(f'{run:2}/{runs}', end='\r')
        dfw = pd.concat([dga.get_chunk(), legit.get_chunk()])
        pred = []
        prob = []
        query_time = []
        results = []

        for domain_to_check in dfw.domain.values:
            st = time.time()
            messages = [{
               "role": "user",
               "content": [{
                    "type": "text",
                    "text": f"Classify the following domain name as DGA or normal: {domain_to_check}"
                }]
            }]

            # Generar prompt en formato texto plano
            text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

            # Tokenizar para entrada al modelo
            inputs = tokenizer([text], return_tensors="pt").to("cuda")

            # Generar la salida (sin streamer para capturar resultado)
            output_ids = model.generate(
                **inputs,
                max_new_tokens=128,
                temperature=1.0,
                top_p=0.95,
                top_k=64,
                do_sample=True
            )

            # Decodificar la salida
            generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)


            # Ejemplo de salida: "The traffic is classified as normal."
            # Parseamos la respuesta:
            if "classified as dga" in generated_text.lower():
                label_value = 1
            elif "classified as normal" in generated_text.lower():
                label_value = 0
            else:
                # Caso no esperado, asignar 0 o manejar de otra forma
                label_value = 0

            pred.append(label_value)
            query_time.append(time.time() - st)

        dfw['pred'] = pred
        # dfw['prob'] = prob  # Si tienes probabilidades, descomenta esta línea
        dfw['query_time'] = query_time
        dfw.to_csv(f'/content/drive/My Drive/results/results_Llama3_FineTuning_docker_{family}_{run}.csv.gz', index=False)


manuelita.gz


In [4]:
import requests
import pandas as pd
import numpy as np
import time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import sys
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import re

families = [
    #'matsnu.gz'#,
    #'suppobox.gz',
    #'charbot.gz',
    #'gozi.gz',
    'manuelita.gz'#,
    #'rovnix.gz',
    #'deception.gz',
    #'nymaim.gz'
]

runs = 15

def fpr_tpr(y, ypred):
    tn, fp, fn, tp = confusion_matrix(y, ypred).ravel()
    fpr = fp / (fp + tn)  # False Positive Rate
    tpr = tp / (tp + fn)  # True Positive Rate (Recall)
    return fpr, tpr

for family in families:
    acc = []
    pre = []
    rec = []
    f1 = []
    fpr = []
    tpr = []
    qt = []
    qts = []
    for run in range(runs):
        df = pd.read_csv(f'/content/drive/My Drive/results/results_Llama3_FineTuning_docker_{family}_{run}.csv.gz')
        y = (df.label == 'dga').astype(int)
        ypred = df.pred
        acc.append(accuracy_score(y, ypred))
        pre.append(precision_score(y, ypred))
        rec.append(recall_score(y, ypred))
        f1.append(f1_score(y, ypred))
        fpr_value, tpr_value = fpr_tpr(y, ypred)
        fpr.append(fpr_value)
        tpr.append(tpr_value)
        qt.append(df.query_time.mean())
        qts.append(df.query_time.std())
#    print(f'Query time: {np.mean(qt):0.5f}+/-{np.mean(qts)}:0.5f')
    print(f'{family.split(".")[0]:15}: acc:{np.mean(acc):0.2f}±{np.std(acc):.3f} f1:{np.mean(f1):0.2f}±{np.std(f1):.3f} pre:{np.mean(pre):0.2f}±{np.std(pre):.3f} rec:{np.mean(rec):0.2f}±{np.std(rec):.3f}  FPR:{np.mean(fpr):0.2f}±{np.std(fpr):.3f} TPR:{np.mean(tpr):0.2f}±{np.std(tpr):.3f} Query time: {np.mean(qt):0.5f}±{np.mean(qts):0.5f}')


manuelita      : acc:0.58±0.028 f1:0.33±0.074 pre:0.83±0.103 rec:0.21±0.057  FPR:0.05±0.029 TPR:0.21±0.057 Query time: 2.18311±1.92626
